In [1]:
from hikyuu.interactive import *

Initialize hikyuu_2.6.8_202510162356_RELEASE_macosx_arm64 ...
2025-10-17 00:14:03.128 [HKU-I] - current python version: 3.12.7 (main.cpp:72)


2025-10-17 00:14:05,037 [INFO] runing in interactive session [<module>] (/Users/fasiondog/workspace/hikyuu/hikyuu/__init__.py:110) [hikyuu::hku_info]
2025-10-17 00:14:05,038 [INFO] running in jupyter [<module>] (/Users/fasiondog/workspace/hikyuu/hikyuu/__init__.py:117) [hikyuu::hku_info]


2025-10-17 00:14:05.042 [HKU-I] - Plugin path: /Users/fasiondog/workspace/hikyuu/hikyuu/plugin (StockManager.cpp:113)
2025-10-17 00:14:06.773 [HKU-I] - Using CLICKHOUSE BaseInfoDriver (BaseInfoDriver.cpp:57)
2025-10-17 00:14:06.793 [HKU-I] - 加载市场信息…… (StockManager.cpp:704)
2025-10-17 00:14:06.802 [HKU-I] - 加载证券类型信息…… (StockManager.cpp:722)
2025-10-17 00:14:06.808 [HKU-I] - 加载证券信息…… (StockManager.cpp:602)
2025-10-17 00:14:06.896 [HKU-I] - 加载权息数据…… (StockManager.cpp:740)
2025-10-17 00:14:07.062 [HKU-I] - 加载板块信息…… (StockManager.cpp:182)
2025-10-17 00:14:07.308 [HKU-I] - 加载K线数据…… (StockManager.cpp:186)
2025-10-17 00:14:07.308 [HKU-I] - 预加载 day K线数据至缓存 (最大数量: 100000)! (StockManager.cpp:231)
2025-10-17 00:14:07.309 [HKU-I] - 0.53 秒数据加载完毕. (StockManager.cpp:193)


In [5]:
# 创建两个因子 ma20, ma60
ma20 = MA(CLOSE(), 20)
ma20.name = 'MA20'

ma60 = MA(CLOSE(), 60)
ma60.name = 'MA60'

# 指定证券列表
stks = [s for s in blocka]

# 指定查询范围，并创建一个等权组合的 MF
query = Query(Datetime(20150101), Datetime(20251017))
mf = MF_EqualWeight([ma20, ma60], stks, query, ref_stk=sm["sh000001"])

In [6]:
# 没有标准化时，获取合成后某日的评分列表
scores = mf.get_scores(Datetime(20251016))
print(scores.to_df())


      证券代码  证券名称        score
0     SH600519  贵州茅台  1460.640867
1     SH603444    吉比特   500.309125
2     SZ002371  北方华创   395.552042
3     BJ920982  锦波生物   300.004475
4     SH605499  东鹏饮料   295.006675
...        ...       ...          ...
3974  BJ872808  曙光数创          NaN
3975  BJ873527    夜光明          NaN
3976  BJ832471  美邦科技          NaN
3977  BJ430478  峆一药业          NaN
3978  SZ000675  ST 银 山          NaN

[3979 rows x 3 columns]


In [7]:
# 添加全局标准化
mf.set_normalize(NORM_Zscore())
scores = mf.get_scores(Datetime(20251016))
print(scores.to_df())

      证券代码  证券名称      score
0     SH600519  贵州茅台  43.150291
1     SH603444    吉比特  14.382820
2     SZ002371  北方华创  11.265517
3     BJ920982  锦波生物   8.427688
4     SH605499  东鹏饮料   8.272032
...        ...       ...        ...
3974  BJ872808  曙光数创        NaN
3975  BJ873527    夜光明        NaN
3976  BJ832471  美邦科技        NaN
3977  BJ430478  峆一药业        NaN
3978  SZ000675  ST 银 山        NaN

[3979 rows x 3 columns]


In [11]:
# 为 ma20 添加行业中性化以及市场中性化(即按市值风格因子中性化)
mf.add_special_normalize("MA20", NORM_Zscore(), category="行业板块", style_inds=[LOG(CLOSE()*LIUTONGPAN())])
scores = mf.get_scores(Datetime(20251016))
print(scores.to_df())

      证券代码  证券名称      score
0     SH600519  贵州茅台  43.114519
1     SH603444    吉比特  14.403012
2     SZ002371  北方华创  11.271072
3     BJ920982  锦波生物   8.407414
4     SH605499  东鹏饮料   8.233678
...        ...       ...        ...
3974  BJ872808  曙光数创        NaN
3975  BJ873527    夜光明        NaN
3976  BJ832471  美邦科技        NaN
3977  BJ430478  峆一药业        NaN
3978  BJ832566    梓橦宫        NaN

[3979 rows x 3 columns]
